# Credit Risk Resampling Techniques

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter


# Read the CSV and Perform Basic Data Cleaning

In [ ]:
columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [ ]:
# Load the data
file_path = Path('LoanStats_2019Q1.csv')
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df.head()

# Split the Data into Training and Testing

In [ ]:
# Create our features
X = df.drop(['loan_status'] ,axis =1)


# Create our target
y = df[['loan_status']]

In [ ]:
#get dummies to convert string into numerical values

X=pd.get_dummies(X)

In [ ]:
X.describe()
# y.describe()

In [ ]:
# Check the balance of our target values
y['loan_status'].value_counts()

In [107]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,
                                                   y,
                                                   random_state=1,
                                                   stratify=y
                                                   )
# X_train.shape()

from sklearn.linear_model import LogisticRegression

classifier = LogisticRegression(solver='lbfgs',
                               max_iter=200,
                               random_state=1)
#prediction 

classifier.fit(X_train,y_train)
y_prediction= classifier.predict(X_test)

#import accuracy score 

from sklearn.metrics import accuracy_score

print(accuracy_score(y_test,y_prediction))



0.9950014530659692


# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

### Naive Random Oversampling

In [ ]:
# Resample the training data with the RandomOversampler
from imblearn.over_sampling import RandomOverSampler
random_over_sample = RandomOverSampler(random_state=1)
X_resampled, y_resampled = random_over_sample.fit_resample(X_train,y_train)

Counter(y_resampled)

In [ ]:
# Train the Logistic Regression model using the resampled data

from sklearn.linear_model import LogisticRegression

log_model = LogisticRegression( solver='lbfgs', max_iter= 100, random_state=1)
log_model.fit(X_resampled,y_resampled)

In [ ]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred = log_model.predict(X_test)
balanced_accuracy_score(y_test,y_pred) 


In [ ]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred)


In [68]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced 

classification_report_imbalanced(y_test,y_pred)
print(classification_report_imbalanced(y_test,y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.63      0.67      0.02      0.65      0.42        87
   low_risk       1.00      0.67      0.63      0.80      0.65      0.43     17118

avg / total       0.99      0.67      0.63      0.80      0.65      0.43     17205



### SMOTE Oversampling

In [70]:
# Resample the training data with SMOTE
from imblearn.over_sampling import SMOTE

X_resampled, y_resampled = SMOTE(random_state=1).fit_resample(X_train,y_train)

Counter(y_resampled)

Counter({'loan_status': 1})

In [73]:
# Train the Logistic Regression model using the resampled data
SMOTE_model = LogisticRegression(solver='lbfgs',random_state=1)
SMOTE_model.fit(X_resampled,y_resampled)

LogisticRegression(random_state=1)

In [74]:
# Calculated the balanced accuracy score
y_pred= SMOTE_model.predict(X_test)

balanced_accuracy_score(y_test,y_pred)

0.6512291961274883

In [76]:
# Display the confusion matrix
confusion_matrix(y_test,y_pred)

array([[   56,    31],
       [ 5841, 11277]])

In [77]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test,y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.64      0.66      0.02      0.65      0.42        87
   low_risk       1.00      0.66      0.64      0.79      0.65      0.42     17118

avg / total       0.99      0.66      0.64      0.79      0.65      0.42     17205



# Undersampling

In this section, you will test an undersampling algorithms to determine which algorithm results in the best performance compared to the oversampling algorithms above. You will undersample the data using the Cluster Centroids algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [86]:
# Resample the data using the ClusterCentroids resampler
# Warning: This is a large dataset, and this step may take some time to complete
from imblearn.under_sampling  import ClusterCentroids 

cluster_centroids = ClusterCentroids(random_state=1)
X_resampled, y_resampled = cluster_centroids.fit_resample(X_train,y_train)
Counter(y_resampled)

Counter({'loan_status': 1})

In [87]:
# Train the Logistic Regression model using the resampled data
under_model = LogisticRegression(solver='lbfgs',random_state=1)
under_model.fit(X_resampled,y_resampled)

LogisticRegression(random_state=1)

In [ ]:
# Calculated the balanced accuracy score
balanced_accuracy_score(y_test,y_pred)

In [88]:
# Display the confusion matrix
y_pred = under_model.predict(X_test)
confusion_matrix(y_test,y_pred)

array([[   56,    31],
       [10258,  6860]])

In [91]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test,y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.64      0.40      0.01      0.51      0.26        87
   low_risk       1.00      0.40      0.64      0.57      0.51      0.25     17118

avg / total       0.99      0.40      0.64      0.57      0.51      0.25     17205



# Combination (Over and Under) Sampling

In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [100]:
# Resample the training data with SMOTEENN
# Warning: This is a large dataset, and this step may take some time to complete

from imblearn.combine import SMOTEENN

resample_smot = SMOTEENN(random_state=1)
X_resampled, y_resampled = resample_smot.fit_resample(X, y)  
Counter(y_resampled)

Counter({'loan_status': 1})

In [101]:
# Train the Logistic Regression model using the resampled data
combo_model = LogisticRegression(solver='lbfgs', random_state=1)
combo_model.fit(X_resampled,y_resampled)

LogisticRegression(random_state=1)

In [103]:
# Calculated the balanced accuracy score
balanced_accuracy_score(y_test,y_pred)

0.5222129559125099

In [105]:
# Display the confusion matrix
y_pred = combo_model.predict(X_test)
confusion_matrix(y_test,y_pred)

array([[   60,    27],
       [ 6910, 10208]])

In [106]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test,y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.69      0.60      0.02      0.64      0.42        87
   low_risk       1.00      0.60      0.69      0.75      0.64      0.41     17118

avg / total       0.99      0.60      0.69      0.74      0.64      0.41     17205

